In [1]:
import glob
import numpy as np

## Boilerplate path hack to give access to full clustered_SNe package
import os, sys
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
from clustered_SNe.analysis.parse import Overview, parse_run
from clustered_SNe.analysis.database_helpers import Simulation_Status, \
                                                    session


/Users/egentry/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/egentry/Dropbox/Data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:38: UserWarning: `session' from database_helpers can only write using 1 process at a time
  UserWarning)


In [2]:
data_dir = "../saved_runs/cluster_parameter_study/"

src_dir = "../src"
scripts_dir = "../scripts/"


# Examples

In [3]:
Simulation_Status.is_converged("ae1f1577-ad61-4f26-99ef-1204be73b417", data_dir)

True

In [4]:
Simulation_Status.is_last_checkpoint_x99("ae1f1577-ad61-4f26-99ef-1204be73b417", data_dir)

True

# Build Statuses from data files

In [5]:
overviews = glob.glob(os.path.join(data_dir, "*overview.dat"))
ids = []
for overview in overviews:
    id = os.path.basename(overview.split("_overview.dat")[0])
    ids.append(id)
ids = np.array(ids)

In [6]:
from multiprocessing import Pool
pool = Pool(4)
ids_and_data_dir = np.array([(id, data_dir) for id in ids])
finished = pool.map(Simulation_Status.mappable_is_last_checkpoint_x99, ids_and_data_dir)
finished = np.array(finished)
converged = pool.map(Simulation_Status.mappable_is_converged, ids_and_data_dir[finished])
converged = np.array(converged)

restart_ids = ids[finished][~converged]

In [7]:
restart_ids

array(['0fdca3cb-3c60-4073-b3d5-33b771786bfc',
       '34ad9744-fd58-43f8-ab50-09b499cf7115',
       '34ba2aec-f27c-4e09-9ffc-8d5e5fc6221d',
       '53c43e6e-1b48-4f96-bd93-1bf18ad5e25c',
       '7f0a5470-12c1-4974-b9d6-50aa90dcb198',
       '99e4f57e-2105-420b-bf78-fbe71b07405f',
       'b9065a50-1513-44a6-a2a3-592035e40a74',
       'd431dd27-ee94-4f72-b81d-e572f9f74266',
       'e6c85ffa-2e11-45b9-8a82-cf6818f91d92',
       'e730e083-9335-4a55-ba55-280c5bd0bcbb'], 
      dtype='<U36')

## Add to database

In [8]:
simulation_statuses = []
for id in ids:
    status = "Unknown"
    if id in ids[finished]:
        status = "Ready"
    if id in ids[finished][converged]:
        status = "Complete"
        
    simulation_status = Simulation_Status(id=id, data_dir=data_dir,
                                          status=status)
    simulation_statuses.append(simulation_status)


for status in simulation_statuses:
    status.add_or_update_to_table()

    

# Restart if ready

In [9]:
for status in session.query(Simulation_Status):
    if status.status is "Ready":
#         status.create_new_batch_file_and_switch_to_running()
        pass

In [10]:
for status in session.query(Simulation_Status):
    print(status.status)

Complete
Complete
Complete
Complete
Unknown
Complete
Complete
Complete
Complete
Complete
Unknown
Ready
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Unknown
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Unknown
Complete
Complete
Complete
Complete
Ready
Ready
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Unknown
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Ready
Complete
Complete
Complete
Unknown
Unknown
Complete
Complete
Complete
Complete
Unknown
Complete
Complete
Unknown
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Unknown
Complete
Complete
Complete
Complete
Complete
Ready
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Ready
Complete
Complete
Complete
Complete
Complete
Complete
Co

# Check statuses from database

In [11]:
statuses = np.array([status.status for status in session.query(Simulation_Status)])
counts = {status : 0 for status in Simulation_Status.possible_statuses}

for status in statuses:
    counts[status] += 1


In [12]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status=="Error"):
    print(simulation_status.id)

In [ ]:
statuses

In [13]:
counts

{'Complete': 166, 'Error': 0, 'Ready': 10, 'Running': 0, 'Unknown': 18}

In [14]:
for status in session.query(Simulation_Status).filter(Simulation_Status.status=="Unknown"):
    print(status.id)


0716f4d7-9411-40d3-b001-4a021cf0c2e1
0edcca3b-6d7f-45ea-9dc2-a7172d68a01d
1cf82e21-095e-4eb6-b554-60a5d9363762
2ec6b483-da85-4781-b5f0-075531580754
417eceab-0a0a-4040-a087-e33d4c38f7dc
560f1259-63e6-40cf-b5ce-40b715490d5c
56d7e5b5-3845-441a-83fa-2996f493501c
5ac6f05a-b4f4-4550-aac9-c6e5327e9f32
5d571036-8dab-4e3f-923d-d058d88f7e96
70f3c240-e3bf-43fa-8bb5-7701167e6b94
a302e259-80cd-425e-b0b0-278ab8778c88
a3f38dec-7696-44cb-b376-7462997d9a97
a4b88f80-50f9-496a-8b0b-8234b05e4dc0
a58d92f5-f1bb-4cfb-88ee-559e6eaeaf4c
cbd8bbe5-389f-438e-b324-7bf3ecc1b46d
e8ed025a-1bd2-44d1-bc79-349db4376822
f4783a27-2b2c-4366-95bb-5e058fcaf869
f8644401-fcff-4a77-bf47-2418e60d19ae


In [15]:
session.commit()

# Shut down SQL session when done

In [16]:
session.close()